In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/dataset.csv')
df

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...
201,2l-c,[*]C1=CC=CC=C1,Ph,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,72.0,0.0,0.0,92.0
202,2m,[*]C1=CSC=C1,C,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,83.0
203,2n,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,53.0
204,2o,NaN,NaN,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,10.0,CH3CN,O2,60,48.0,0.0,0.0,25.0


# 前処理
1. 欠損値削除(R1-,R2,organocatalyst,yield)
2. under = Ar削除(1個)
3. R2 = Ph削除(3個)
4. 触媒促進剤使ってるデータも削除
5. rtを23度に置換
6. カテゴリ値をダミー変数に変換

In [ ]:
# 基質、触媒、収率で欠損がある場合は削除
df.dropna(subset=['R1-', 'R2', 'organocatalyst', 'yield'], inplace=True)

In [ ]:
print('欠損値合計', df.isnull().sum())
na_row = df.isnull().any(axis=1)
df.loc[na_row, :]

欠損値合計 entry                   0
R1-                     0
R2                      0
organocatalyst          0
organocatalyst(mol%)    0
solvent                 0
under                   0
temp(℃)                 0
time(h)                 0
Cu(OAc)2(mol%)          0
AcOH(mol%)              0
yield                   0
dtype: int64


,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield


In [ ]:
df.head()

,entry,R1-,R2,organocatalyst,organocatalyst(mol%),solvent,under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),yield
0,1,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,24.0,0.0,0.0,81.0
1,2,[*]C1=CC=CC=C1,H,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,toluene,O2,90,12.0,0.0,0.0,14.0
2,3,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,74.0
3,4,[*]C1=CC=CC=C1,H,OC1=CC(C)=CC=C1C(O)=O,5.0,toluene,O2,90,12.0,0.0,0.0,15.0
4,5,[*]C1=CC=CC=C1,H,OC1=CC(OC)=CC=C1C(O)=O,5.0,toluene,O2,90,24.0,0.0,0.0,90.0


In [ ]:
# organocatalyst = '-'のデータ削除
df = df[df['organocatalyst'] != '-']
df['organocatalyst'] == '-'

0      False
1      False
2      False
3      False
4      False
       ...  
199    False
200    False
201    False
202    False
205    False
Name: organocatalyst, Length: 153, dtype: bool

In [ ]:
# Arを削除(1 data)
df = df[df['under'] != 'Ar']
df['under'].value_counts()

O2     108
air     44
Name: under, dtype: int64

In [ ]:
# R2 = Ph削除(3 data)
df = df[df['R2'] != 'Ph']
df['R2'].value_counts()

H    97
C    52
Name: R2, dtype: int64

In [ ]:
# 触媒促進剤データ削除
df = df[df['Cu(OAc)2(mol%)'] == 0]
df = df[df['AcOH(mol%)'] == 0]
print(df.columns)
print(df.shape)

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'Cu(OAc)2(mol%)',
       'AcOH(mol%)', 'yield'],
      dtype='object')
(121, 12)


In [ ]:
# 触媒カラム削除
df.drop(columns=['Cu(OAc)2(mol%)','AcOH(mol%)'],inplace=True)

df.columns

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)',
       'solvent', 'under', 'temp(℃)', 'time(h)', 'yield'],
      dtype='object')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
# rtを23度に置換
df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"
print(df['temp(℃)'].value_counts())

23    64
90    48
80     7
60     2
Name: temp(℃), dtype: int64


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_92445/2448397563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp(℃)'][df['temp(℃)'] == 'rt'] = "23"


In [ ]:
# カテゴリ値をダミー変数に変換
df = pd.get_dummies(df, columns=['R2','under','solvent'])
df.head(5)

,entry,R1-,organocatalyst,organocatalyst(mol%),temp(℃),time(h),yield,R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_DMF,solvent_MeCN,solvent_MeOH,solvent_toluene
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,24.0,81.0,0,1,1,0,0,0,0,0,1
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,5.0,90,12.0,14.0,0,1,1,0,0,0,0,0,1
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,24.0,74.0,0,1,1,0,0,0,0,0,1
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,5.0,90,12.0,15.0,0,1,1,0,0,0,0,0,1
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,5.0,90,24.0,90.0,0,1,1,0,0,0,0,0,1


In [ ]:
# 重複確認
# check_columns = df.columns.drop(['entry', '収率(%)'])

# # 説明変数のみでみた時の重複削除
# _df = df.drop(columns='entry').drop_duplicates(subset=check_columns).reset_index(drop=True)
# X = _df.drop(columns=['収率(%)'])
# y = _df['収率(%)']

In [ ]:
# 重複確認
def isDuplicated(df):
    check_columns = df.columns.drop(['entry', 'yield'])
    return df.duplicated(subset=check_columns).value_counts()

In [ ]:
isDuplicated(df)

False    121
dtype: int64

In [ ]:
df.to_csv('./data/origin.csv')

# スケーリング

In [12]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
def scaling(df, scaling_columns):
    scaler = StandardScaler()
    scaler.fit(df[scaling_columns])
    df_scaled = pd.concat([df.drop(columns=scaling_columns), 
                               pd.DataFrame(scaler.transform(df[scaling_columns]),index=df.index , columns=scaling_columns)
                          ],axis=1, join='inner')
    print(df_scaled.shape)
    return df_scaled

In [ ]:
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)']
df_scaled = scaling(df, scaling_columns)

ValueError: could not convert string to float: 'rt'

In [ ]:
df_scaled.to_csv('./data/origin_scaled.csv')

# 記述子
https://qiita.com/ottantachinque/items/d7332bcf67b5844f5be3
### morganフィンガープリントの作成（Extended Connectivity Fingerprint）
子からある距離にある部分構造を数え上げていく，「circular substructures」と呼ばれるタイプのフィンガープリントです．radiusの値で距離を設定します．いわゆるECFP（Extended Connectivity Fingerprint）フィンガープリントに相当するものですが，探索距離の定義が異なる点に注意が必要です．
### morgan featureフィンガープリントの作成(Functional Connectivity Fingerprint）
FCFP（Functional Connectivity Fingerprint）様のフィンガープリントを用いたい場合にはuseFeatures=Trueを設定します．

### MACCS
全部で166の部分構造についての有無を調べ上げたもので，RDKit内の情報保持のために1ビット使っているため全部で167ビットのフィンガープリントになります．部分構造を有する場合は1が，ない場合は0が格納されています．

### RDkit記述子
一定の結合数に相当する原子と結合種類を格納する方法で，事前に部分構造を用意する必要がないことから，より柔軟に分子構造を表現することが可能です．

### MinHashフィンガープリント
「A probabilistic molecular fingerprint for big data settings」という論文で報告されている，ECFPと同様にCircular型のフィンガープリントになります

### Avalonフィンガープリント

### アトムペアフィンガープリントとトポロジカル二面角フィンガープリント
原子のアトムタイプとその最小結合距離を記録していきます．
アトムタイプには
原子の種類
結合する重原子の数
π電子数
https://future-chem.com/ap-dp-fingerprint/

### ドナーアクセプターペアフィンガープリント
ドナーアクセプターペアフィンガープリントでは「アトムタイプの情報」について

水素結合供与体
水素結合受容体
といった一般性のある「化学的性質」を情報として用いているフィンガープリントです

### mordred記述子の作成
https://github.com/mordred-descriptor


MolLogP : Crippenらによる原子ベースのLogP指標(LogPは水/オクタノール分配係数）
MolWt : 分子量
NumHAcceptors : 水素結合アクセプターの数
NumHDonors : 水素結合ドナーの数
NumHeteroatoms : へテロ元素の数
NumRotatableBonds : 回転可能な結合数
FractionCSP3 : 全炭素数におけるsp3炭素の割合
TPSA : トポロジカル極性表面積


In [88]:
import pandas as pd
df_scaled = pd.read_csv('./data/origin_scaled.csv', index_col=0)
df_scaled

,entry,R1-,organocatalyst,yield,R2_C,R2_H,under_O2,under_air,solvent_CH3CN,solvent_DMF,solvent_MeOH,solvent_toluene,organocatalyst(mol%),temp(℃),time(h)
0,1,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,81.0,0,1,1,0,0,0,0,1,-0.599526,1.120490,-0.171686
1,2,[*]C1=CC=CC=C1,ClC1=CC=C(C(O)=O)C(O)=C1,14.0,0,1,1,0,0,0,0,1,-0.599526,1.120490,-0.801200
2,3,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,74.0,0,1,1,0,0,0,0,1,-0.599526,1.120490,-0.171686
3,4,[*]C1=CC=CC=C1,OC1=CC(C)=CC=C1C(O)=O,15.0,0,1,1,0,0,0,0,1,-0.599526,1.120490,-0.801200
4,5,[*]C1=CC=CC=C1,OC1=CC(OC)=CC=C1C(O)=O,90.0,0,1,1,0,0,0,0,1,-0.599526,1.120490,-0.171686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2i,[*]C1=CC(Br)=CC=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,95.0,1,0,1,0,1,0,0,0,0.746348,-0.935602,2.346371
117,2j,[*]C1=CC=CC(OC)=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,89.0,1,0,1,0,1,0,0,0,0.746348,-0.935602,1.087343
118,2k,[*]C1=CC(C)=C(C)C=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,99.0,1,0,1,0,1,0,0,0,0.746348,-0.935602,2.346371
119,2m,[*]C1=CSC=C1,O=C1C(C=C(OC)C(C(C)(C)C)=C1)=O,83.0,1,0,1,0,1,0,0,0,0.746348,0.199852,1.087343


In [14]:
BIT = 2048

In [89]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Avalon import pyAvalonTools
from mordred import Calculator, descriptors
from rdkit.Chem.AtomPairs.Sheridan import GetBPFingerprint
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem import rdMHFPFingerprint
encoder = rdMHFPFingerprint.MHFPEncoder()

def toFinger(df, columns, finger_method):
    df_copy = df.drop(columns=columns)
    for column in columns:
        fingerprints = toFingerFromSmiles(df[column],finger_method)
        if finger_method in ['mordred_2d','mordred_3d']:
            fingerprints_df = fingerprints.add_suffix('_'+column).set_axis(df.index, axis='index')
        else:
            column_names = list(map(lambda x: str(x)+'_'+column, range(len(fingerprints[0]))))
            fingerprints_df =  pd.DataFrame(fingerprints,index=df.index, columns=column_names)
        
        # scaling必要
        # if finger_method in ['mordred_2d','mordred_3d', 'minhash']:
        #     fingerprints_df = scaling(fingerprints_df, fingerprints_df.columns)
        fingerprints_df = fingerprints_df[fingerprints_df.columns[~fingerprints_df.isnull().any()]]
        df_copy = pd.merge(df_copy, fingerprints_df, left_index=True, right_index=True)
    return df_copy

def toFingerFromSmiles(series, method):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    if (method == 'mordred_2d'):
        # scaling必要
        calc_2D = Calculator(descriptors, ignore_3D=True) #2D記述子
        fingerprints = calc_2D.pandas(mols,quiet=False)
        for column in fingerprints.columns:
            if fingerprints[column].dtypes == object:
                fingerprints[column] = fingerprints[column].values.astype(np.float32)
        return fingerprints
    elif (method == 'mordred_3d'):
        # scaling必要
        calc_3D = Calculator(descriptors, ignore_3D=False) #3D記述子
        fingerprints = calc_3D.pandas(mols,quiet=False)
        for column in fingerprints.columns:
            if fingerprints[column].dtypes == object:
                fingerprints[column] = fingerprints[column].values.astype(np.float32)
        return fingerprints
    else:
        fingerprints = []
        for mol_idx, mol in enumerate(mols):
            try:
                # listに直してる。
                if (method == 'morgan'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT)]
                elif (method == 'morgan_feature'):
                    fingerprint = [x for x in AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT,useFeatures=True)]
                elif (method == 'maccs'):
                    fingerprint = [x for x in AllChem.GetMACCSKeysFingerprint(mol)]
                elif (method == 'rdkit'):
                    fingerprint = [x for x in Chem.RDKFingerprint(mol)]
                elif (method == 'minhash'):
                    # scaling必要
                    fingerprint = [x for x in encoder.EncodeMol(mol)]
                elif (method == 'avalon'):
                    fingerprint = [x for x in pyAvalonTools.GetAvalonFP(mol)]
                elif (method == 'atom'):
                    fingerprint = [x for x in Pairs.GetAtomPairFingerprintAsBitVect(mol)]
                elif (method == 'donor'):
                    print('donor')
                    fingerprint = [x for x in GetBPFingerprint(mol)]
                else:
                    print('method error')
                fingerprints.append(fingerprint)
            except Exception as e:
                print(e)
                print("Error", mol_idx)
                break
        return fingerprints

In [16]:
finger_methods = [
    # 'morgan',
    # 'morgan_feature',
    # 'maccs',
    # 'rdkit',
    # 'minhash',
    'avalon',
    # 'atom', # 重い
    #'donor', # 重い
    'mordred_2d',
    # 'mordred_3d',
]

# 記述子変換 ⇒ 分散が0 or 80%異常が同じ値の情報を削除
説明しきれていないため共線性の恐れがある
VarianceThresholdで特徴選択

In [17]:
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

def removeNoImpactData(df):
    X = df.drop(columns=['entry','yield'])
    y = df['yield']
    # ベルヌーイ分布に基づく分散の閾値は0.16
    select = VarianceThreshold(threshold=(.8 * (1 - .8)))
    select.fit_transform(X.values)
    selected_columns = X.columns[select.get_support()]
    selected_X = pd.DataFrame(X[selected_columns], columns=selected_columns)

    # 相関が0.95以上のカラムを削除
    X_new = deleteHighCorrColumn(selected_X)
    
    df_new = pd.concat([
        y,
        X_new,
    ],axis=1)
    print('before_shape: ', df.shape)
    print('new_shape: ', df_new.shape)
    return df_new

def deleteHighCorrColumn(df):
    threshold = 0.95
    df_corr = df.corr()
    df_corr = abs(df_corr)
    columns = df_corr.columns

    # 対角線の値を0にする
    for i in range(0, len(columns)):
        df_corr.iloc[i, i] = 0

    while True:
        columns = df_corr.columns
        max_corr = 0.0
        query_column = None
        target_column = None

        df_max_column_value = df_corr.max()
        max_corr = df_max_column_value.max()
        query_column = df_max_column_value.idxmax()
        target_column = df_corr[query_column].idxmax()

        if max_corr < threshold:
            # しきい値を超えるものがなかったため終了
            break
        else:
            # しきい値を超えるものがあった場合
            delete_column = None
            saved_column = None

            # その他との相関の絶対値が大きい方を除去
            if sum(df_corr[query_column]) <= sum(df_corr[target_column]):
                delete_column = target_column
                saved_column = query_column
            else:
                delete_column = query_column
                saved_column = target_column

            # 除去すべき特徴を相関行列から消す（行、列）
            df_corr.drop([delete_column], axis=0, inplace=True)
            df_corr.drop([delete_column], axis=1, inplace=True)

    return df[df_corr.columns]

def boruta(X,y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    corr_list = []
    for n in range(10000):
        shadow_features = np.random.rand(X_scaled.shape[0]).T
        corr = np.corrcoef(X_scaled, shadow_features,rowvar=False)[-1]
        corr = abs(corr[corr < 0.95])
        corr_list.append(corr.max())

    corr_array = np.array(corr_list)
    perc = 100 * (1-corr_array.max())

    # RandomForestRegressorでBorutaを実行
    rf = RandomForestRegressor(n_jobs=-1, max_depth=5)
    feat_selector = BorutaPy(
        rf,
        n_estimators='auto',
        verbose=2,
        alpha=0.05, # 有意水準
        max_iter=100, # 試行回数
        perc = perc, #ランダム生成変数の重要度の何％を基準とするか
        random_state=1
    )
    feat_selector.fit(X_scaled, y.values)

    # 選択された特徴量を確認
    selected = feat_selector.support_
    print('選択された特徴量の数: %d' % np.sum(selected))
    print(selected)
    print(X.columns[selected])

    # 選択した特徴量で学習
    X_selected = X[X.columns[selected]]
    return X_selected
    

# finger print変換
- method1 全てのデータを使用

In [ ]:
# 1. 全てのデータを使用
finger_columns = ['R1-', 'organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df_method1 = toFinger(df_scaled, finger_columns,finger_method)
    df_method1.to_csv(f'./data/method1/{finger_method}_noRemoved.csv')
    removeNoImpactData(df_method1).to_csv(f'./data/method1/{finger_method}.csv')

In [ ]:
# method boruta
finger_columns = ['R1-', 'organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df = pd.read_csv(f'./data/method1/{finger_method}.csv',index_col=0)
    X = df.drop(columns=['yield'])
    y = df['yield']
    X_boruta = boruta(X,y)
    print(X_boruta.columns)
    pd.concat([y,X_boruta],axis=1).to_csv(f'./data/boruta/{finger_method}.csv')

morgan_feature
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	27
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	13
Tentative: 	7
Rejected: 	7
Iteration: 	9 / 100
Confirmed: 	13
Tentative: 	7
Rejected: 	7
Iteration: 	10 / 100
Confirmed: 	13
Tentative: 	7
Rejected: 	7
Iteration: 	11 / 100
Confirmed: 	13
Tentative: 	7
Rejected: 	7
Iteration: 	12 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	9
Iteration: 	13 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	9
Iteration: 	14 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	9
Iteration: 	15 / 100
Confirmed: 	14
Tentative: 	4
Rejected: 	9
Iteration: 	16 / 100
Confirmed: 	14
Tentative: 	4

# 構造予測データ

In [90]:
# toFinger(df_scaled,['R1-','organocatalyst'],'avalon').to_csv('./data/origin/avalon.csv')
# toFinger(df_scaled,['R1-','organocatalyst'],'mordred_2d').to_csv('./data/origin/mordred.csv')




100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:26<00:00,  4.49it/s]


In [70]:
r1 = pd.DataFrame({'SMILES':['CC1=CC=CC=C1']})
toFinger(r1,['SMILES'],'avalon').to_csv('./data/structure_creation/avalon_r1.csv')
toFinger(r1,['SMILES'],'mordred_2d').to_csv('./data/structure_creation/mordred_r1.csv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10it/s]


In [65]:
# structure_creation
FINGER_COLUMN=['SMILES']
finger_method = 'mordred_2d'
df = pd.read_csv('./data/structure_creation/generated.csv',index_col=0)
split_num = 10000
for i in range(round(df.shape[0]/split_num)):
    start = i * split_num
    df_structure = toFinger(df.iloc[start:start+split_num,:],FINGER_COLUMN,finger_method)
    display(df_structure)
    df_structure.to_csv(f'./data/structure_creation/{finger_method}_{i}.csv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [44:10<00:00,  3.77it/s]


,ABC_SMILES,ABCGG_SMILES,nAcid_SMILES,nBase_SMILES,SpAbs_A_SMILES,SpMax_A_SMILES,SpDiam_A_SMILES,SpAD_A_SMILES,SpMAD_A_SMILES,LogEE_A_SMILES,...,SRW10_SMILES,TSRW10_SMILES,MW_SMILES,AMW_SMILES,WPath_SMILES,WPol_SMILES,Zagreb1_SMILES,Zagreb2_SMILES,mZagreb1_SMILES,mZagreb2_SMILES
name,,,,,,,,,,,,,,,,,,,,,
structure 1,16.774613,16.428494,2,0,26.654527,2.525710,5.051419,26.654527,1.158892,4.019937,...,10.132494,57.297715,323.136887,7.344020,1024,41,112.0,133.0,11.861111,5.333333
structure 2,15.739337,15.714480,1,1,26.650378,2.523090,5.046180,26.650378,1.211381,3.974516,...,10.107856,56.092220,313.079766,8.461615,908,41,106.0,128.0,11.000000,5.333333
structure 3,16.136456,15.877342,4,0,25.372814,2.532513,5.065027,25.372814,1.153310,3.978161,...,10.136383,56.189454,312.048132,9.177886,895,40,108.0,129.0,11.611111,5.027778
structure 4,13.480117,13.438945,1,1,22.798999,2.489657,4.979314,22.798999,1.199947,3.827304,...,9.917834,52.227259,262.131742,7.084642,626,34,90.0,108.0,9.027778,4.694444
structure 5,18.024283,17.626712,2,0,26.541703,2.572653,5.145306,26.541703,1.105904,4.075030,...,10.368761,59.062848,336.193674,6.465263,1095,44,124.0,147.0,13.513889,5.138889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
structure 9996,16.624783,16.519108,2,1,27.406515,2.535560,5.071120,27.406515,1.191588,4.020298,...,10.173018,57.380945,326.075015,8.812838,1014,43,112.0,135.0,11.861111,5.444444
structure 9997,15.291450,14.923487,1,0,24.753774,2.503124,5.006247,24.753774,1.178751,3.930144,...,10.026280,54.815284,291.121906,7.661103,830,37,102.0,121.0,10.750000,4.888889
structure 9998,16.581560,16.117979,1,0,24.435855,2.537268,5.074535,24.435855,1.110721,3.990044,...,10.238709,56.574450,308.162374,6.699182,890,39,114.0,134.0,12.402778,4.666667


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [58:05<00:00,  2.87it/s]


,ABC_SMILES,ABCGG_SMILES,nAcid_SMILES,nBase_SMILES,SpAbs_A_SMILES,SpMax_A_SMILES,SpDiam_A_SMILES,SpAD_A_SMILES,SpMAD_A_SMILES,LogEE_A_SMILES,...,SRW10_SMILES,TSRW10_SMILES,MW_SMILES,AMW_SMILES,WPath_SMILES,WPol_SMILES,Zagreb1_SMILES,Zagreb2_SMILES,mZagreb1_SMILES,mZagreb2_SMILES
name,,,,,,,,,,,,,,,,,,,,,
structure 10001,16.942470,16.409722,1,0,26.205852,2.534096,5.068191,26.205852,1.139385,4.023768,...,10.184787,57.452292,322.214409,6.079517,1030,40,114.0,134.0,11.951389,5.194444
structure 10002,14.543903,14.514844,3,1,23.294266,2.516673,5.033345,23.294266,1.164713,3.884559,...,10.052812,53.742643,283.069202,8.577855,693,37,98.0,118.0,10.500000,4.666667
structure 10003,11.141620,10.474205,1,0,17.324259,2.371357,4.742715,17.324259,1.154951,3.597969,...,9.465835,46.704770,209.105193,6.970173,362,21,72.0,81.0,7.416667,3.333333
structure 10004,16.027066,15.854732,2,0,26.134522,2.527207,5.054414,26.134522,1.187933,3.978332,...,10.137967,56.205614,312.059365,9.178217,910,41,108.0,130.0,11.611111,5.111111
structure 10005,17.372330,17.007265,1,0,28.695188,2.528265,5.056529,28.695188,1.195633,4.060219,...,10.158479,58.456926,333.132471,7.747267,1158,43,116.0,138.0,12.111111,5.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
structure 19996,12.325124,12.146431,1,1,20.482588,2.454782,4.909564,20.482588,1.204858,3.722024,...,9.781037,49.693515,238.095357,7.680495,470,29,82.0,97.0,8.527778,4.000000
structure 19997,11.577577,11.604428,1,1,19.022919,2.467936,4.935872,19.022919,1.188932,3.665584,...,9.807692,48.594399,227.079373,7.830323,389,29,78.0,94.0,8.277778,3.777778
structure 19998,15.141620,15.117322,2,0,24.966761,2.520236,5.040472,24.966761,1.188893,3.930549,...,10.080922,54.920803,298.068867,8.516253,796,39,102.0,123.0,10.750000,5.000000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [51:53<00:00,  3.21it/s]


,ABC_SMILES,ABCGG_SMILES,nAcid_SMILES,nBase_SMILES,SpAbs_A_SMILES,SpMax_A_SMILES,SpDiam_A_SMILES,SpAD_A_SMILES,SpMAD_A_SMILES,LogEE_A_SMILES,...,SRW10_SMILES,TSRW10_SMILES,MW_SMILES,AMW_SMILES,WPath_SMILES,WPol_SMILES,Zagreb1_SMILES,Zagreb2_SMILES,mZagreb1_SMILES,mZagreb2_SMILES
name,,,,,,,,,,,,,,,,,,,,,
structure 20001,15.291450,14.931148,1,0,24.793791,2.503849,5.007699,24.793791,1.180657,3.930144,...,10.026722,54.815727,298.105253,7.643724,828,37,102.0,121.0,10.750000,4.888889
structure 20002,14.693733,14.360549,1,0,23.222569,2.495786,4.991572,23.222569,1.161128,3.884126,...,9.986955,53.620112,283.105587,7.651502,716,35,98.0,116.0,10.500000,4.555556
structure 20003,15.141620,15.037423,1,0,25.386721,2.514279,5.028558,25.386721,1.208891,3.930539,...,10.070822,54.906845,293.064785,9.158275,814,39,102.0,123.0,10.750000,5.000000
structure 20004,13.808287,13.549534,1,1,22.302097,2.484215,4.968431,22.302097,1.173795,3.831496,...,9.918819,52.296888,262.131742,7.084642,632,33,92.0,109.0,9.638889,4.444444
structure 20005,17.080370,16.751687,3,0,25.948930,2.553562,5.107123,25.948930,1.128214,4.027768,...,10.258957,57.646566,324.120903,7.537695,992,42,116.0,138.0,12.562500,5.083333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
structure 29996,18.079437,17.575575,1,0,29.741695,2.530771,5.061541,29.741695,1.189668,4.098782,...,10.172407,59.582622,351.168188,7.023364,1288,44,120.0,142.0,12.361111,5.916667
structure 29997,14.890100,14.653549,1,0,22.786857,2.527649,5.055297,22.786857,1.139343,3.893287,...,10.142544,54.046538,284.089603,7.891378,702,37,102.0,122.0,11.201389,4.388889
structure 29998,8.910910,8.632017,1,0,13.407823,2.358294,4.716589,13.407823,1.117319,3.383453,...,9.324115,42.775979,166.062994,7.548318,184,17,58.0,66.0,6.055556,2.666667


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5621/5621 [29:25<00:00,  3.18it/s]


,ABC_SMILES,ABCGG_SMILES,nAcid_SMILES,nBase_SMILES,SpAbs_A_SMILES,SpMax_A_SMILES,SpDiam_A_SMILES,SpAD_A_SMILES,SpMAD_A_SMILES,LogEE_A_SMILES,...,SRW10_SMILES,TSRW10_SMILES,MW_SMILES,AMW_SMILES,WPath_SMILES,WPol_SMILES,Zagreb1_SMILES,Zagreb2_SMILES,mZagreb1_SMILES,mZagreb2_SMILES
name,,,,,,,,,,,,,,,,,,,,,
structure 30001,13.917677,13.436504,1,0,22.253112,2.477339,4.954678,22.253112,1.171216,3.831278,...,9.893387,52.244801,266.163043,6.491782,646,32,92.0,108.0,9.638889,4.361111
structure 30002,14.500680,14.142349,1,0,20.295556,2.529497,5.058994,20.295556,1.068187,3.849844,...,10.142465,52.981323,266.151809,6.491508,590,33,100.0,117.0,11.041667,3.888889
structure 30003,15.441280,14.882613,1,0,24.103954,2.491466,4.982932,24.103954,1.147807,3.929748,...,9.979661,54.720381,295.178358,6.416921,828,35,102.0,119.0,10.750000,4.777778
structure 30004,15.487817,15.320318,1,0,24.298042,2.535510,5.071019,24.298042,1.157050,3.938895,...,10.176678,55.227435,292.105922,7.894755,792,39,106.0,127.0,11.451389,4.722222
structure 30005,16.774613,16.428494,2,0,26.648762,2.524709,5.049418,26.648762,1.158642,4.019937,...,10.131698,57.296920,321.084852,8.449601,1024,41,112.0,133.0,11.861111,5.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
structure 35617,12.325124,12.105839,1,1,20.138021,2.446632,4.893264,20.138021,1.184589,3.722013,...,9.766808,49.674253,240.074621,8.278435,476,29,82.0,97.0,8.527778,4.000000
structure 35618,15.487817,15.328636,1,1,23.903211,2.534041,5.068083,23.903211,1.138248,3.938895,...,10.175536,55.226293,295.153206,7.027457,790,39,106.0,127.0,11.451389,4.722222
structure 35619,14.963281,14.865729,1,1,25.224636,2.508570,5.017141,25.224636,1.201173,3.926347,...,10.025395,54.752631,292.105922,7.894755,810,38,100.0,120.0,10.138889,5.138889


In [108]:
# structure creatioin 2
FINGER_COLUMN=['SMILES']
finger_method = ['avalon','mordred_2d']
df = pd.read_csv('./data/structure_creation2/smiles.csv',index_col=0)
# エラー出るrowを削除
drop_row = [
    0,6,7,8,88,89,121,196,216,227,318,339,367,415,472,473,500,501,502,506,507,508,509,510,511,512,513,514,515,516,517,519,520,522,532,
    637,690,715,729,769,778,836,837,903,978,
 ]
err_cnt=0
df_list = pd.DataFrame()
for i in range(df.shape[0]):
    df_split = df.iloc[i:i+1,:]
    print(i,'回目')
    df_ = toFinger(df_split,FINGER_COLUMN,'mordred_2d')
    try:
        print(df_.shape)
        df_[['ATS1s_SMILES', 'AATS0p_SMILES', 'AATSC0i_SMILES','GATS4s_SMILES', 'BalabanJ_SMILES']]
        df_list = pd.concat([df_list,df_])
    except Exception as e:
        print(e)
        err_cnt+=1
        continue
    df_list.to_csv(f'./data/structure_creation2/mordred.csv')
    print(err_cnt)

0 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.36s/it]


/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
(1, 1178)
"['ATS1s_SMILES', 'GATS4s_SMILES'] not in index"
1 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.03s/it]


(1, 1444)
1
2 回目


...

(1, 1448)
45
998 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.39s/it]


(1, 1445)
45
999 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]


(1, 1443)
45


In [110]:
# structure creation 3
FINGER_COLUMN=['SMILES']
finger_method = ['avalon','mordred_2d']
df = pd.read_csv('./data/structure_creation3/smiles.csv',index_col=0)

toFinger(df,['SMILES'],'avalon').to_csv('./data/structure_creation3/avalon.csv')
# エラー出るrowを削除
# drop_row = [
#     0,6,7,8,88,89,121,196,216,227,318,339,367,415,472,473,500,501,502,506,507,508,509,510,511,512,513,514,515,516,517,519,520,522,532,
#     637,690,715,729,769,778,836,837,903,978,
#  ]
err_cnt=0
df_list = pd.DataFrame()
for i in range(df.shape[0]):
    df_split = df.iloc[i:i+1,:]
    print(i,'回目')
    df_ = toFinger(df_split,FINGER_COLUMN,'mordred_2d')
    try:
        print(df_.shape)
        df_[['ATS1s_SMILES', 'AATS0p_SMILES', 'AATSC0i_SMILES','GATS4s_SMILES', 'BalabanJ_SMILES']]
        df_list = pd.concat([df_list,df_])
    except Exception as e:
        print(e)
        err_cnt+=1
        continue
    df_list.to_csv(f'./data/structure_creation3/mordred.csv')
    print(err_cnt)

0 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.17it/s]


(1, 1389)
0
1 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.50it/s]


(1, 1129)
"['ATS1s_SMILES', 'GATS4s_SMILES'] not in index"
2 回目


...

(1, 1438)
15
89 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.01s/it]


(1, 1334)
15
90 回目


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]


(1, 1430)
15


# method2 
- R1 [*]C1=CC=CC=C1 
- R2_H = 1
- solvent toluene

In [ ]:
df = pd.read_csv('./data/origin.csv', index_col=0)

In [ ]:
for column in df.columns:
    print(df[column].value_counts())


1       7
3       7
2       7
4       6
6       6
5       5
7       5
13      4
12      4
9       4
8       4
10      3
11      3
2a      3
2b      3
2c      3
2d      3
2e      3
2f      3
2g      3
2j      2
2k      2
2m      2
2i      2
2h      2
3f      1
2h-c    1
3q      1
3o      1
3n      1
3m      1
3l      1
3k      1
3j      1
3i      1
3h      1
3g      1
3e      1
2l      1
3d      1
24      1
22      1
21      1
20      1
19      1
14      1
1p      1
1o      1
1n      1
2p      1
Name: entry, dtype: int64
[*]C1=CC=CC=C1                  49
[*]C1=CC=C(OC)C=C1              15
[*]C1=CC=C(C)C=C1                7
[*]C1=CC=C(Cl)C=C1               4
[*]C1=CC(OCO2)=C2C=C1            3
[*]C1=CC=C(F)C=C1                3
[*]C1=CC=CO1                     3
[*]C1=CC(Cl)=CC=C1               2
[*]C1=CC=C([N+]([O-])=O)C=C1     2
[*]C1=CC=CC(Cl)=C1               2
[*]C1=C(C)C=CC=C1                2
[*]C1=CC=CS1                     2
[*]C1=C(C=CC=C2)C2=CC=C1         2
[*]C1=CC=C(C(F)(F)F

In [ ]:
df_ = df[df['R1-'] == '[*]C1=CC=CC=C1']
df_ = df_[df_['R2_H'] == 1]
df_ = df_[df_['solvent_toluene'] == 1]
df_.shape

(27, 16)

In [ ]:
df_dropped = df_.drop(columns=['R1-', 'R2_C', 'R2_H', 'solvent_CH3CN', 'solvent_DMF', 'solvent_MeCN', 'solvent_MeOH', 'solvent_toluene', 'under_O2', 'under_air','organocatalyst(mol%)', 'temp(℃)'])




In [ ]:
scaling_columns = ['time(h)']
df_dropped_scaled = scaling(df_dropped, scaling_columns).reset_index(drop=True)

(27, 4)


In [ ]:
# 2. 限りなく除去
finger_columns = ['organocatalyst']
for finger_method in finger_methods:
    print(finger_method)
    df_method2 = toFinger(df_dropped_scaled, finger_columns,finger_method)
    removeNoImpactData(df_method2).to_csv(f'./data/method2/{finger_method}.csv')

morgan
before_shape:  (27, 1027)
new_shape:  (27, 113)
morgan_feature
before_shape:  (27, 1027)
new_shape:  (27, 84)
maccs
before_shape:  (27, 170)
new_shape:  (27, 73)
rdkit
before_shape:  (27, 2051)
new_shape:  (27, 1082)
minhash
(27, 2048)
before_shape:  (27, 2051)
new_shape:  (27, 2050)
avalon
before_shape:  (27, 515)
new_shape:  (27, 266)
mordred_2d


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:11<00:00,  2.38it/s]
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 

(27, 1613)
before_shape:  (27, 1616)
new_shape:  (27, 1148)
mordred_3d


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:08<00:00,  3.30it/s]


(27, 1826)
before_shape:  (27, 1829)
new_shape:  (27, 1148)


/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:985: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:990: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/utils/extmath.py:1020: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:104: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/Users/watanabeyuitou/opt/anaconda3/envs/chem/lib/python3.9/site-packages/sklearn/feature_selection/_variance_threshold.py:112: RuntimeWarning: All-NaN